*Downloading and cleaning data on flights and airports*

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import urllib.request as req

# Download data on flights and airports

### Flights

Data comes from http://stat-computing.org/dataexpo/2009/the-data.html. Flights data relates to the year 2007 as it's the last year with complete information.

In [2]:
# Flights
filename = '2007.csv.bz2'
url = 'http://stat-computing.org/dataexpo/2009/'+str(filename)
%time req.urlretrieve(url,filename)
%time Flights = pd.read_csv(filename, compression='bz2')

Flights = Flights[["Year","Month","DayofMonth","DayOfWeek","DepTime","UniqueCarrier","Origin","Dest","Distance"]]

# Save un-cleaned to pickle: too big for Github, as >100 MB
# Flights.to_pickle("Flights.pkl")

print(Flights.shape)
print(Flights.columns)

Wall time: 17.4 s
Wall time: 50 s
(7453215, 9)
Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance'],
      dtype='object')


***Unique journeys for price scraping***

In [3]:
scrape_data = Flights.copy()
scrape_data["Journey"] = scrape_data["Origin"]+scrape_data["Dest"]
scrape_data = scrape_data.drop_duplicates(subset = "Journey", keep = "first")
scrape_data.head(2)
# scrape_data.to_pickle("scrape_data.pkl")

,Year,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Journey
0,2007,1,1,1,1232.0,WN,SMF,ONT,389,SMFONT
1,2007,1,1,1,1918.0,WN,SMF,PDX,479,SMFPDX


### Airports

Data comes from https://openflights.org/data.html

In [14]:
# Load data
airports_raw = pd.read_csv('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat', header = None) #load data

# Assign column names
airports_raw.columns = ["Airport_id","Name","City","Country","3DigitId","4DigitId","Lat","Lon","Altitude_ft","Timezone","DST","TZ","Type","Source"] 

# Pick relevant columns for analysis
airports_df = airports_raw[["Airport_id","Name","City","Country","3DigitId","Lat","Lon","Altitude_ft"]]

# Pickle the dataframe for future use
airports_df.to_pickle("Airports.pkl")

print('Shape:', airports_df.shape)
airports_df.head(2)

Shape: (7698, 8)


,Airport_id,Name,City,Country,3DigitId,Lat,Lon,Altitude_ft
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,-6.08169,145.391998,5282
1,2,Madang Airport,Madang,Papua New Guinea,MAG,-5.20708,145.789001,20


In [5]:
# Data on US airports from stat-computing.org
# filename = 'airports.csv'
# url = 'http://stat-computing.org/dataexpo/2009/'+str(filename)
# req.urlretrieve(url,filename)
# Airports = pd.read_csv(filename)

# print('Shape:', Airports.shape)
# Airports.head(2)

# Data cleaning for flights data

### Count number of flights between airports

In [6]:
# Select variables from which to create network. Must contain two columns of nodes, since 
# nx.from_pandas_edgelist() views each row as an edge. Can also contains attributes, e.g. Distance. 
FlightsNx = Flights[["Origin","Dest","Distance"]]

# Take counts (i.e. how many origin to destination flights in the dataset)
counts = pd.DataFrame(Flights.groupby(["Origin","Dest"])["Year"].count())
counts.columns = ["count"]
FlightsNx = FlightsNx.merge(counts, how = 'left', on = ["Origin","Dest"])

# Drop duplicates 
FlightsNx = FlightsNx.drop_duplicates()

# Count number of take-offs from origin: 
orig_takeoff = pd.DataFrame(FlightsNx.groupby(["Origin"])["count"].sum())
orig_takeoff.columns = ["origin_takeoff"]
# Merge to network dataset: 
FlightsNx = FlightsNx.merge(orig_takeoff, how = "left", on = "Origin")

# Count number of take-offs from dest
dest_takeoff = pd.DataFrame(FlightsNx.groupby(["Origin"])["count"].sum())
dest_takeoff.columns = ["dest_takeoff"]
FlightsNx = FlightsNx.merge(dest_takeoff, how = "left", left_on = "Dest", right_on = "Origin")

# Count number of landings at destination
dest_lands = pd.DataFrame(FlightsNx.groupby(["Dest"])["count"].sum())
dest_lands.columns = ["dest_landing"]
FlightsNx = FlightsNx.merge(dest_lands, how = "left", on = "Dest")

orig_lands = pd.DataFrame(FlightsNx.groupby(["Dest"])["count"].sum())
orig_lands.columns = ["orig_landing"]
FlightsNx = FlightsNx.merge(orig_lands, how = "left", left_on = "Origin", right_on = "Dest")

# Construct totals (flights)
FlightsNx["Origin_flights"] = FlightsNx["origin_takeoff"]+FlightsNx["orig_landing"]
FlightsNx["Destination_flights"] = FlightsNx["dest_takeoff"]+FlightsNx["dest_landing"]

# Drop auxiliary vars
FlightsNx.drop(["origin_takeoff","dest_takeoff","dest_landing","orig_landing"], axis = 1, inplace = True)

FlightsNx.head(2)

,Origin,Dest,Distance,count,Origin_flights,Destination_flights
0,SMF,ONT,389,4093,116158,83287.0
1,SMF,PDX,479,2721,116158,117237.0


### Implement as network
For function documentation, see:  
https://networkx.github.io/documentation/stable/reference/generated/networkx.convert_matrix.from_pandas_edgelist.html?highlight=from_pandas_edgelist#networkx.convert_matrix.from_pandas_edgelist

In [7]:
Network = nx.from_pandas_edgelist(FlightsNx, source = "Origin", target = "Dest", edge_attr = True)

### Degree distribution

In [8]:
# docs: https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.degree_centrality.html#networkx.algorithms.centrality.degree_centrality 
# Find degree: 
degree_dict = nx.algorithms.centrality.degree_centrality(Network) 
degree_values = list(degree_dict.values())
# Create dataframe
degrees = pd.DataFrame(degree_values)
degrees.columns = ["degree"]
degrees["degree_unnormalized"] = degrees["degree"]*(len(Network.nodes)-1)
degrees["airport"] = degree_dict.keys()

# Save to pickle for descriptive
degrees.to_pickle("Degrees.pkl")

In [9]:
# Add degree to FlightsNx data
FlightsNx = FlightsNx.merge(degrees, how = "left", left_on = "Origin", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = {"degree": "origin_degree", "degree_unnormalized": "origin_degree_unn"})

FlightsNx = FlightsNx.merge(degrees, how = "left", left_on = "Dest", right_on = "airport")
FlightsNx = FlightsNx.drop("airport", axis = 1)
FlightsNx = FlightsNx.rename(columns = {"degree": "dest_degree", "degree_unnormalized": "dest_degree_unn"})

FlightsNx.head(2)

,Origin,Dest,Distance,count,Origin_flights,Destination_flights,origin_degree,origin_degree_unn,dest_degree,dest_degree_unn
0,SMF,ONT,389,4093,116158,83287.0,0.142395,44.0,0.129450,40.0
1,SMF,PDX,479,2721,116158,117237.0,0.142395,44.0,0.139159,43.0


In [10]:
# Save cleaned to pickle
FlightsNx.to_pickle("FlightsNx.pkl") # only 0.4 MB